In [ ]:
# --- 🧭 SETUP ---
# This notebook cleans Monthly_Transportation_Statistics.csv
# Keep only transportation activity metrics (remove safety, spending, sales, and macro data)
# Also removes all data before 1975 (based on year)

import os
import zipfile
from pathlib import Path

import pandas as pd
from IPython.display import HTML, display

# ✅ Robust download helper (works in Colab; shows a link fallback)
def download_with_fallback(path, *, zip_if_large_mb=50):
    p = Path(path).resolve()
    if not p.exists():
        print(f"❌ File not found: {p}")
        return

    size_mb = p.stat().st_size / (1024 * 1024)
    target_path = p
    if size_mb >= zip_if_large_mb:
        zip_path = p.with_suffix(p.suffix + ".zip")
        with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
            zf.write(p, arcname=p.name)
        target_path = zip_path
        print(f"📦 File was large ({size_mb:.1f} MB). Created archive: {zip_path.name}")

    print(f"➡️ Preparing download for: {target_path.name} ({target_path.stat().st_size/1024:.1f} KB)")
    try:
        from google.colab import files as colab_files
        colab_files.download(str(target_path))
        print("✅ Download triggered in your browser. If nothing happens, enable pop-ups for Colab.")
    except Exception:
        display(HTML(f'<a href="{target_path.name}" download style="font-size:16px;">⬇️ Click to download {target_path.name}</a>'))

# --- 🔹 1. Upload your file ---
try:
    from google.colab import files
    uploaded = files.upload()
    INPUT_CSV = list(uploaded.keys())[0]
except Exception:
    print("Not running in Colab. Set INPUT_CSV to your local CSV path.")
    INPUT_CSV = "Monthly_Transportation_Statistics.csv"

OUTPUT_CSV = "Monthly_Transportation_Statistics_cleaned.csv"

# --- 🔹 2. Load dataset ---
df = pd.read_csv(INPUT_CSV)
print("Original shape:", df.shape)

# --- 🔹 3. Helper to check column keywords ---
orig_cols = df.columns.tolist()
lower_map = {c: c.lower() for c in orig_cols}
def col_has_any(col: str, needles_lower):
    col_l = lower_map[col]
    return any(n in col_l for n in needles_lower)

# --- 🔹 4. Remove safety, spending, sales, and price/cost columns ---
exclude_1 = [
    "Fatalit", "Safety", "Spending", "sales", "Cost Index", "Price", "Construction"
]
exclude_1_lower = [s.lower() for s in exclude_1]
keep_cols_1 = [c for c in orig_cols if not col_has_any(c, exclude_1_lower)]
df1 = df[keep_cols_1]

# --- 🔹 5. Remove macroeconomic & employment data ---
exclude_2 = [
    "Employment", "Unemployment", "Labor Force", "GDP", "Gross Domestic Product",
    "Participation Rate", "Real", "Amtrak On-time", "Person Crossings"
]
exclude_2_lower = [s.lower() for s in exclude_2]
keep_cols_2 = [c for c in df1.columns if not col_has_any(c, exclude_2_lower)]
clean_df = df1[keep_cols_2].copy()

# --- 🔹 6. Parse dates for time series analysis ---
if "Date" in clean_df.columns:
    clean_df["Date_parsed"] = pd.to_datetime(clean_df["Date"], errors="coerce", infer_datetime_format=True)

# If parsing fails, try extracting numeric year manually
if clean_df["Date_parsed"].isna().all():
    print("⚠️ Warning: Could not parse Date column automatically. Trying to extract year from text.")
    clean_df["Year"] = clean_df["Date"].astype(str).str.extract(r"(\d{4})")
    clean_df["Year"] = pd.to_numeric(clean_df["Year"], errors="coerce")
else:
    clean_df["Year"] = clean_df["Date_parsed"].dt.year

# --- 🔹 7. Remove data before 1975 ---
before_filter = clean_df.shape[0]
clean_df = clean_df[clean_df["Year"] >= 1975].reset_index(drop=True)
after_filter = clean_df.shape[0]
print(f"\n📅 Removed {before_filter - after_filter} rows before 1975.")
print("Years remaining:", sorted(clean_df["Year"].dropna().unique())[:5], "... →", sorted(clean_df["Year"].dropna().unique())[-5:])

# Reorder columns to keep Date columns first
front = [c for c in ["Index", "Date", "Date_parsed", "Year"] if c in clean_df.columns]
other = [c for c in clean_df.columns if c not in front]
clean_df = clean_df[front + other]

# --- 🔹 8. Save intermediate cleaned dataset ---
clean_df.to_csv(OUTPUT_CSV, index=False)
print("\n✅ Cleaning complete!")
print("After filtering shape:", clean_df.shape)

print("\nKept columns:")
for c in clean_df.columns:
    print("-", c)

# --- 🔹 9. Download intermediate cleaned file ---
print("\n⬇️ Downloading intermediate cleaned file...")
download_with_fallback(OUTPUT_CSV)

# --- 🔹 10. Drop empty rows (dates) with no data ---
before_rows = clean_df.shape[0]
value_cols = [c for c in clean_df.columns if "Date" not in c and "Index" not in c and c not in ["Year"]]

# Drop rows where all activity columns are NaN
lean_df = clean_df.dropna(subset=value_cols, how="all").reset_index(drop=True)

# Drop rows where Date_parsed is missing or invalid
if "Date_parsed" in lean_df.columns:
    lean_df = lean_df.dropna(subset=["Date_parsed"]).reset_index(drop=True)

after_rows = lean_df.shape[0]
removed = before_rows - after_rows

print(f"\n🧹 Removed {removed} rows that contained no data or invalid dates.")
print(f"Remaining rows: {after_rows}")

# --- 🔹 11. Save final cleaned dataset ---
OUTPUT_CSV_FINAL = "Monthly_Transportation_Statistics_CLEAN_FINAL.csv"
lean_df.to_csv(OUTPUT_CSV_FINAL, index=False)
print(f"\n✅ Final cleaned dataset saved as: {OUTPUT_CSV_FINAL}")

# --- 🔹 12. Download final cleaned file ---
print("\n⬇️ Downloading final cleaned dataset...")
download_with_fallback(OUTPUT_CSV_FINAL)


Saving Monthly_Transportation_Statistics.csv to Monthly_Transportation_Statistics (10).csv
Original shape: (884, 136)

📅 Removed 336 rows before 1975.
Years remaining: [np.int32(1975), np.int32(1976), np.int32(1977), np.int32(1978), np.int32(1979)] ... → [np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020)]

✅ Cleaning complete!
After filtering shape: (548, 40)

Kept columns:
- Index
- Date
- Date_parsed
- Year
- U.S. Airline Traffic - Total - Seasonally Adjusted
- U.S. Airline Traffic - International - Seasonally Adjusted
- U.S. Airline Traffic - Domestic - Seasonally Adjusted
- Transit Ridership - Other Transit Modes - Adjusted
- Transit Ridership - Fixed Route Bus - Adjusted
- Transit Ridership - Urban Rail - Adjusted
- Freight Rail Intermodal Units
- Freight Rail Carloads
- Highway Vehicle Miles Traveled - All Systems
- Highway Vehicle Miles Traveled - Total Rural
- Highway Vehicle Miles Traveled - Other Rural
- Highway Vehicle Miles Traveled - Rural Othe

/tmp/ipython-input-784771527.py:78: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  clean_df["Date_parsed"] = pd.to_datetime(clean_df["Date"], errors="coerce", infer_datetime_format=True)
/tmp/ipython-input-784771527.py:78: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  clean_df["Date_parsed"] = pd.to_datetime(clean_df["Date"], errors="coerce", infer_datetime_format=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download triggered in your browser. If nothing happens, enable pop-ups for Colab.

🧹 Removed 1 rows that contained no data or invalid dates.
Remaining rows: 547

✅ Final cleaned dataset saved as: Monthly_Transportation_Statistics_CLEAN_FINAL.csv

⬇️ Downloading final cleaned dataset...
➡️ Preparing download for: Monthly_Transportation_Statistics_CLEAN_FINAL.csv (110.6 KB)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download triggered in your browser. If nothing happens, enable pop-ups for Colab.
